## 여러개 파일 연속 처리 테스트

In [20]:
import os

from app.services.analysis.image_preprocessor import ImagePreprocessor, Settings as IPS
from app.services.ocr.paddle_ocr import PaddleOCRService
from app.services.analysis.line_preprocessor import LinePreprocessor, Settings as LPSettings
from app.services.analysis.reference.unit_lexicon import get_unit_lexicon
import app.services.analysis.lab_table_extractor as lte
import importlib
import app.services.analysis.reference.code_lexicon as cl

lte = importlib.reload(lte)
LabTableExtractor = lte.LabTableExtractor

Settings = lte.Settings

ip = ImagePreprocessor(IPS(debug=True))

ocr = PaddleOCRService(lang="korean")

unit_lex = get_unit_lexicon()  # 전역 유닛 렉시콘(캐시)

lp = LinePreprocessor(LPSettings(
    order="x_left",
    alpha=0.7,
    debug=True,
    normalize_units_first_pass=True,  # 분리 직후 1차 단위 정규화 
), unit_lexicon=unit_lex)


cl = importlib.reload(cl)
code_lexicon = cl.get_code_lexicon()  # 캐시 사용 (참조 데이터 변경 없을 때 추천)

extractor = LabTableExtractor(
    settings=Settings(use_llm=True, debug=True),  # 정책 미달 시에만 LLM 백업 사용
    lexicon=code_lexicon,
    resolver=cl.resolve_code,  # ← 중요: 최신 resolve_code를 명시 주입
    api_key=os.getenv("OPENAI_API_KEY"),
    llm_model="gpt-4.1-mini",
)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)

In [21]:
img_paths = [
    os.path.abspath('notebooks/ocr/assets/images/20241121_0.png'),
    os.path.abspath('notebooks/ocr/assets/images/20241113.png'),
    os.path.abspath('notebooks/ocr/assets/images/20241121_1.png'),
]
    
print(f"[전처리+OCR] 총 {len(img_paths)}개 이미지 처리 시작")

# 추출 결과 누적 리스트
results = []

for p in img_paths:
    print(f"\n[작업 시작] {p}")
    try:
        with open(p, 'rb') as f:
            _raw_bytes = f.read()

        # 클래스 기반 전처리 실행 (디버그 로그 포함: 데스큐 각도, 용량 변화 등)
        _pre_bytes = ip.process_bytes(_raw_bytes, debug=True)
        print(f"✅ ImagePreprocessor 로 전처리 완료: {p} (bytes: {len(_pre_bytes) if _pre_bytes else 'None'})")

        # OCR 실행
        ocr_result = ocr.run_ocr_from_bytes(_pre_bytes)

        # 라인 정렬 및 값/단위 분리
        lined = lp.extract_and_group_lines(ocr_result[0])

        result = extractor.extract(lined)
        # 누적
        results.append(result)
    except Exception as e:
        print(f"⚠️ 처리 실패: {p} - {e}")

[전처리+OCR] 총 3개 이미지 처리 시작

[작업 시작] /home/aidan/work/meow-chat/notebooks/ocr/assets/images/20241121_0.png
🏥 이미지 전처리 시작 (ImagePreprocessor)
🔁 EXIF 회전 교정 적용 시도
📏 원본 이미지 크기: (1179, 1568)
🧰 실행 단계: 1 → upscale_min_resolution
[pipeline] -> upscale_min_resolution(min_long_edge=1920)
[pipeline]    size (1179, 1568) → (1444, 1920), mode RGBA → RGBA
💾 저장 형식: PNG(optimize=True)
📐 최종 이미지 크기: (1444, 1920)
🔄 크기 변화: (1179, 1568) → (1444, 1920)
💾 용량 변화: 2,424,766 bytes → 2,911,659 bytes (120.1%)
✅ 전처리 완료
✅ ImagePreprocessor 로 전처리 완료: /home/aidan/work/meow-chat/notebooks/ocr/assets/images/20241121_0.png (bytes: 2911659)
📐 최종 이미지 크기: (1444, 1920)
🔄 크기 변화: (1179, 1568) → (1444, 1920)
💾 용량 변화: 2,424,766 bytes → 2,911,659 bytes (120.1%)
✅ 전처리 완료
✅ ImagePreprocessor 로 전처리 완료: /home/aidan/work/meow-chat/notebooks/ocr/assets/images/20241121_0.png (bytes: 2911659)
[DEBUG] tokens_with_geometry: count=147
[
  {
    "text": "2024-11-21",
    "confidence": 0.9997645616531372,
    "y_top": 0,
    "y_bottom": 37,
    

In [22]:
import json

print("병합 전")
print(json.dumps(results, ensure_ascii=False, indent=2))

병합 전
[
  {
    "hospital_name": "",
    "client_name": "",
    "patient_name": "",
    "inspection_date": "2024-11-21",
    "header_shape": "name-reference-result-unit",
    "tests": [
      {
        "code": "T.Protein",
        "unit": "g/dL",
        "reference_min": 5.7,
        "reference_max": 7.8,
        "value": 6.1
      },
      {
        "code": "Albumin",
        "unit": "g/dL",
        "reference_min": 2.3,
        "reference_max": 3.5,
        "value": 2.9
      },
      {
        "code": "Globulin",
        "unit": "g/dL",
        "reference_min": 2.7,
        "reference_max": 5.2,
        "value": 3.2
      },
      {
        "code": "T.Bilirubin",
        "unit": "mg/dL",
        "reference_min": 0.1,
        "reference_max": 0.4,
        "value": 0.1
      },
      {
        "code": "ALB/GLOB",
        "unit": "UNKNOWN",
        "reference_min": 0.6,
        "reference_max": 1.1,
        "value": 0.9
      },
      {
        "code": "ANIONGAP",
        "unit": "UNKNO

In [24]:
# results 병합 로직을 함수로 캡슐화하여 JSON 문자열을 반환
import json

def merge_results_to_json(results):
    """
    주어진 results 리스트를 병합 규칙에 따라 결합하고 최종 병합 결과를
    JSON 문자열(ensure_ascii=False, indent=2)로 반환합니다.

    규칙:
    1) tests가 비어있는 result 제거
    2) 연속된 result 중 선순위에 inspection_date가 존재하고 후순위는 비어 있으며,
       날짜 외 메타(hospital_name, client_name, patient_name, header_shape)가 동일하면 병합
    3) 각 result의 tests에서 (code, unit) 기준으로 중복 제거 (선순위 보존)
    """
    def _is_empty_tests(item: dict) -> bool:
        tests = item.get('tests')
        return not isinstance(tests, list) or len(tests) == 0

    def _empty_date(item: dict) -> bool:
        d = item.get('inspection_date')
        if d is None:
            return True
        if isinstance(d, str) and d.strip() == '':
            return True
        return False

    def _non_empty_date(item: dict) -> bool:
        return not _empty_date(item)

    def _norm_blank(v):
        if v is None:
            return ''
        try:
            s = str(v).strip()
        except Exception:
            s = ''
        return s

    def _meta_equal(a: dict, b: dict) -> bool:
        # 날짜 제외, 나머지 메타데이터 동일 여부 검사 (공백은 동일로 처리)
        keys = ['hospital_name', 'client_name', 'patient_name', 'header_shape']
        return all(_norm_blank(a.get(k)) == _norm_blank(b.get(k)) for k in keys)

    def _dedup_tests_by_code_unit(tests: list) -> list:
        """(code, unit) 문자열이 같은 항목은 후순위를 제거하고 선순위를 보존."""
        if not isinstance(tests, list):
            return []
        seen = set()
        out = []
        for t in tests:
            if not isinstance(t, dict):
                continue
            key = (_norm_blank(t.get('code')), _norm_blank(t.get('unit')))
            if key in seen:
                continue
            seen.add(key)
            out.append(t)
        return out

    # 1) tests가 비어있는 result 제거
    _filtered = [r for r in results if not _is_empty_tests(r)]

    # 2) 연속 병합 규칙 적용
    _merged = []
    for cur in _filtered:
        if not _merged:
            _merged.append(dict(cur))
            continue
        prev = _merged[-1]
        if _non_empty_date(prev) and _empty_date(cur) and _meta_equal(prev, cur):
            prev_tests = prev.get('tests')
            if not isinstance(prev_tests, list):
                prev_tests = []
                prev['tests'] = prev_tests
            cur_tests = cur.get('tests') or []
            if isinstance(cur_tests, list):
                prev_tests.extend(cur_tests)
            # 후순위 cur는 추가하지 않음
        else:
            _merged.append(dict(cur))

    # 3) (code, unit) 기준 중복 제거
    for item in _merged:
        item['tests'] = _dedup_tests_by_code_unit(item.get('tests', []))

    # JSON 문자열로 반환
    return json.dumps(_merged, ensure_ascii=False, indent=2)

In [25]:
# 병합 결과 JSON 출력 (사용 예)
print("병합 후")
print(merge_results_to_json(results))

병합 후
[
  {
    "hospital_name": "",
    "client_name": "",
    "patient_name": "",
    "inspection_date": "2024-11-21",
    "header_shape": "name-reference-result-unit",
    "tests": [
      {
        "code": "T.Protein",
        "unit": "g/dL",
        "reference_min": 5.7,
        "reference_max": 7.8,
        "value": 6.1
      },
      {
        "code": "Albumin",
        "unit": "g/dL",
        "reference_min": 2.3,
        "reference_max": 3.5,
        "value": 2.9
      },
      {
        "code": "Globulin",
        "unit": "g/dL",
        "reference_min": 2.7,
        "reference_max": 5.2,
        "value": 3.2
      },
      {
        "code": "T.Bilirubin",
        "unit": "mg/dL",
        "reference_min": 0.1,
        "reference_max": 0.4,
        "value": 0.1
      },
      {
        "code": "ALB/GLOB",
        "unit": "UNKNOWN",
        "reference_min": 0.6,
        "reference_max": 1.1,
        "value": 0.9
      },
      {
        "code": "ANIONGAP",
        "unit": "UNKNO